In [13]:
import agate
text = agate.Text()
tester = agate.TypeTester(force={
        'WFO': text,
        'BEGIN_LOCATION': text,
        'EVENT_NARRATIVE': text,
        'EPISODE_ID': text,
        'CZ_FIPS': text,
        'EVENT_NARRATIVE': text
    },limit=300)
nebraskastorms = agate.Table.from_csv('nebraska.csv', column_types=tester)

CastError: Can not convert value 3 to bool.

In [ ]:
print(nebraskastorms)

For this, things seem to be good. Case #, because it has number in the name, might tempt you to think it's a number, but it's not anything you're going to do math on, so it's text. Dates appear to be dates, things that aren't numbers appear to be text, and the two dollar figures appear to be counted as numbers. That's a good start.

The second smell we can find in Agate is Missing Data. We can do that through a series of Group By and Count steps. Let's start with Incident Codes.

In [3]:
codes = unlcrimes.group_by('Incident Code')
code_counts = codes.aggregate([
    ('count', agate.Length())
])
code_counts = code_counts.order_by('Incident Code')

In [4]:
code_counts.print_table()

|------------------------------------------------------+--------|
|  Incident Code                                       | count  |
|------------------------------------------------------+--------|
|  ACCIDENTS - INJURY                                  |    25  |
|  ACCIDENTS - INJURY H&R                              |     1  |
|  ACCIDENTS - P.D. H&R NOT REPORTABLE                 |   327  |
|  ACCIDENTS - P.D. H&R REPORTABLE                     |    15  |
|  ACCIDENTS - P.D. NOT REPORTABLE                     |   213  |
|  ACCIDENTS - P.D. REPORTABLE                         |    71  |
|  ALCOHOL - BUY/PROCURE FOR MINORS                    |    20  |
|  ALCOHOL - CONSUMPTION IN PUBLIC                     |    12  |
|  ALCOHOL - DRUNK                                     |   543  |
|  ALCOHOL - DWI                                       |   558  |
|  ALCOHOL - MINOR IN POSSESSION                       |   954  |
|  ALCOHOL - OPEN CONTAINER                            |    20  |
|  ALCOHOL

And there's that last one. It's blank. There is missing data. But it's only 1 record of thousands, so it's not going to matter in the grand scheme. If that had been larger, we would have to do some more reporting.

Let's now look at **Gaps in Data**. It's been my experience that gaps in data often have to do with time, so let's first look at crimes by year, so we can see if there's a year with big jumps in reported crime. You'd expect the number to change, but not by huge amounts. Huge change would indicate, more often than not, that the data is missing. 

In [5]:
crimes_with_years = unlcrimes.compute([
    ('reported_year', agate.Formula(agate.Text(), lambda row: '%s' % row['Reported'].year))
])

In [6]:
years = crimes_with_years.group_by('reported_year')
year_counts = years.aggregate([
    ('count', agate.Length())
])
year_counts.print_table()

|----------------+--------|
|  reported_year | count  |
|----------------+--------|
|  2010          |   185  |
|  2011          | 1,948  |
|  2012          | 1,881  |
|  2013          | 2,052  |
|  2014          | 1,920  |
|  2015          | 1,750  |
|----------------+--------|


So, something doesn't look right here. 185 crimes in 2010? Not likely. And what about 2015? A ~200 incident drop wouldn't be *that* unheard of -- look at 2012 to 2013 -- but I'm suspicious. Did we get the whole of 2015? Let's check.

In [7]:
crimes_with_monthyears = unlcrimes.compute([
    ('reported_monthyear', agate.Formula(agate.Text(), lambda row: '%s/%s' % (row['Reported'].month, row['Reported'].year)))
])

In [8]:
monthyears = crimes_with_monthyears.group_by('reported_monthyear')
monthyear_counts = monthyears.aggregate([
    ('count', agate.Length())
])
monthyear_counts.print_bars('reported_monthyear', 'count', width=60)

reported_monthyear count
11/2010               56 ▓░░░░░░                            
12/2010              129 ▓░░░░░░░░░░░░░░░                   
1/2011               157 ▓░░░░░░░░░░░░░░░░░░                
2/2011               168 ▓░░░░░░░░░░░░░░░░░░░               
3/2011               157 ▓░░░░░░░░░░░░░░░░░░                
4/2011               184 ▓░░░░░░░░░░░░░░░░░░░░░             
5/2011               130 ▓░░░░░░░░░░░░░░░                   
6/2011               124 ▓░░░░░░░░░░░░░░                    
7/2011               137 ▓░░░░░░░░░░░░░░░░                  
8/2011               183 ▓░░░░░░░░░░░░░░░░░░░░░             
9/2011               242 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░       
10/2011              175 ▓░░░░░░░░░░░░░░░░░░░░              
11/2011              179 ▓░░░░░░░░░░░░░░░░░░░░              
12/2011              112 ▓░░░░░░░░░░░░░                     
1/2012               194 ▓░░░░░░░░░░░░░░░░░░░░░░            
2/2012               147 ▓░░░░░░░░░░░░░░░░░                 

So, it appears we were right to suspect 2015. Did 2015 end magically in November? Nope. We have incomplete data. 

## Assignment

What about Location and Building? Or Stolen and Damaged? Is there missing data? Is there wrong data? How normalized is that data? Are there outliers? What steps in Agate should you take to find out? 

Group UNL locations together, count them and then order them by alphabetical order to find similar entries

In [11]:
locations = unlcrimes.group_by('Location')
locations_counts = locations.aggregate([
    ('count', agate.Length())
])
locations_sorted = locations_counts.order_by('Location')
locations_sorted.print_bars('Location', 'count', width=200)

Location                                                                         count
( )                                                                                  1 ▓                                                                                                                
(AIRPORT)                                                                            1 ▓                                                                                                                
(ALLEY)                                                                              1 ▓                                                                                                                
(BANKS & FINANCIAL)                                                                  1 ▓                                                                                                                
(CHURCH/ETC.)                                                                        1 ▓                     

Group UNL locations together, count them and then order them by alphabetical order to find similar entries

In [12]:
buildings = unlcrimes.group_by('Building')
building_counts = buildings.aggregate([
    ('count', agate.Length())
])
buildings_sorted = building_counts.order_by('Building')
buildings_sorted.print_bars('Building', 'count', width=100)

Building                                       count
1001 Y Street                                      4 ▓                                              
1101 Y St                                          1 ▓                                              
1102 T St                                          3 ▓                                              
1220 N. 14th                                      48 ▓░                                             
14th & Avery Parking Garage                       77 ▓░                                             
14th and W st.                                     2 ▓                                              
1631 Court (Track Office)                          2 ▓                                              
16th and Y st.                                     1 ▓                                              
17 & R Parking Garage                            136 ▓░░                                            
1740 Vine, American Math Competition  

Add agatestats to do standard deviations

In [14]:
import agatestats

Patch agatestats so it can use agate table methods

In [15]:
agatestats.patch()

Filter out unl crimes without a stolen amount

In [26]:
cleanStolen = unlcrimes.where(lambda row: row['Stolen'] != None) 

Find the outliers from cleaned data

In [27]:
stolen_outliers = cleanStolen.stdev_outliers('Stolen')

Find the highest stolen amounts

In [36]:
stolen_outliers_sorted = stolen_outliers.order_by('Stolen', reverse=True)

Select only the stolen amount and description to print out

In [37]:
stolen_print = stolen_outliers_sorted.select(['Stolen', 'Description'])

Print out stolen table

In [38]:
stolen_print.print_table()

|-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|    Stolen | Description                                                                                                                                                                                                                                                                                                                      |
|-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Filter out UNL crimes without stolen amounts

In [55]:
cleanDamaged = unlcrimes.where(lambda row: row['Damaged'] != None)

Trying to fix blank damaged amounts in table

In [66]:
cleanDamagedAmounts = cleanDamaged.where(lambda row: row['Damaged'] > 1)

Compute outliers from damaged amounts higher than 1

In [67]:
damaged_outliers = cleanDamagedAmounts.stdev_outliers('Damaged')

Sort those amounts to see highest damage first

In [68]:
damaged_outliers_sorted = stolen_outliers.order_by('Damaged', reverse=True)

Only grab fields of Damaged and Description

In [69]:
damaged_print = damaged_outliers_sorted.select(['Damaged', 'Description'])

Print out damaged tables

In [70]:
damaged_print.print_table()

|-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   Damaged | Description                                                                                                                                                                                                                                                                                                                      |
|-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------